In [2]:
# Import libraries
import pandas as pd
import json
import requests
import numpy as np

In [3]:
# Set the API URL for NBA teams
dk_nba_team_url = 'https://sportsbook-us-ny.draftkings.com//sites/US-NY-SB/api/v5/eventgroups/42648?format=json'

# Get team data from the API
dk_nba_team_data = requests.get(dk_nba_team_url).json()['eventGroup']

In [4]:
# Select columns for games dataframe
nba_game_df_cols = ['eventId', 'nameIdentifier', 'startDate', 'teamShortName1', 'teamShortName2', 'eventStatus.state']

# Create dataframe of games available
nba_game_df = pd.json_normalize(dk_nba_team_data['events'])[nba_game_df_cols]

# Convert startDate to datetime, then convert to EST by subtracting 5 hours
nba_game_df['startDate'] = pd.to_datetime(nba_game_df['startDate'])
nba_game_df['startDate'] = nba_game_df['startDate'] - pd.Timedelta(hours = 5)

# Rename eventStatus.state to gameState
nba_game_df.rename(columns = {'eventStatus.state': 'gameState'}, inplace = True)

# Only take games that have not started
nba_game_df = nba_game_df[nba_game_df['gameState'] == 'NOT_STARTED']   

# Create away column from first word of nameIdentifier, trim whitespace
nba_game_df['awayTeamName'] = nba_game_df['nameIdentifier'].str.split(' @ ').str[0].str.strip()
# Create home column from second word of nameIdentifier, trim whitespace
nba_game_df['homeTeamName'] = nba_game_df['nameIdentifier'].str.split(' @ ').str[1].str.strip()

# Add NBA type column
nba_game_df['leagueSlug'] = 'NBA'

# Rename teamShortName1 and teamShortName2
nba_game_df.rename(columns = {'teamShortName1': 'awayTeamSlug', 'teamShortName2': 'homeTeamSlug'}, inplace = True)

# Drop nameIdentifier and gameState
nba_game_df.drop(columns = ['nameIdentifier', 'gameState'], inplace = True)

## --- INSERT/UPDATE nba_game_df in SQL, should call table something like dk_events --- ##

In [6]:
# Intitialize number of games to loop through
today_game_len = len(nba_game_df)

## Get offers df from offerCategories and offerSubcategoryDescriptors
# Game lines first level
nba_team_offer_cats = [x for x in dk_nba_team_data['offerCategories'] if x['name'] == 'Game Lines'][0]['offerSubcategoryDescriptors']

# Filter for normal game lines ID (total/point spread/moneyline)
nba_team_game_lines = [x for x in nba_team_offer_cats if x['subcategoryId'] == 4511][0]

# Get number of offers to loop through
offers_to_loop = len(nba_team_game_lines['offerSubcategory']['offers'])

In [9]:
def create_nba_team_odds_df(nba_team_game_lines, event_ind):

    ### Function to create NBA team odds dataframe for a given event index
    ## Args:
    # nba_team_game_lines (df): nba_team_game_lines that has been filtered to subcategoryId 4511 from DK API
    # event_ind (int): index of event to get odds for
    ## Returns:
    # event_odds_df (df): dataframe of available odds for passed event index

    # Try to get event dataframe
    try:
        # Get event dataframe
        event_df = pd.json_normalize(
            nba_team_game_lines['offerSubcategory']['offers'][event_ind])
    except:
        # If live, return empty dataframe
        return pd.DataFrame()

    ## Get various odds
    # Spread
    try:
        spread_lines = pd.json_normalize(
            event_df['outcomes'][event_df[event_df['label'] == 'Spread'].index[0]])[['oddsAmerican', 'label', 'line']]
        spread_lines['oddType'] = 'Spread'
    except:
        # Error -> return empty dataframe
        spread_lines = pd.DataFrame()

    # Moneyline
    try:
        ml_lines = pd.json_normalize(
            event_df['outcomes'][event_df[event_df['label'] == 'Moneyline'].index[0]])[['oddsAmerican', 'label']]
        ml_lines['oddType'] = 'Moneyline'
    except:
        # Error -> return empty dataframe
        ml_lines = pd.DataFrame()

    # Total
    try:
        total_lines = pd.json_normalize(
            event_df['outcomes'][event_df[event_df['label'] == 'Total'].index[0]])[['oddsAmerican', 'label', 'line']]
        total_lines['oddType'] = 'Total'
    except:
        # Error -> return empty dataframe
        total_lines = pd.DataFrame()
    ##
    
    # Try to construct event_odds_df
    try:
        # Get event id from event_df
        event_id = event_df['eventId'][0]

        # Combine all odds into one dataframe
        event_odds_df = pd.concat(
            [spread_lines, ml_lines, total_lines], 
            axis=0
        )

        # Add event id to dataframe
        event_odds_df['eventId'] = event_id
    except:
        # Error -> return empty dataframe
        event_odds_df = pd.DataFrame()

    return event_odds_df

In [10]:
# Map offers_to_loop to create_nba_team_odds_df, join nba_game_df
nba_team_odds_df = pd.concat(
    [create_nba_team_odds_df(nba_team_game_lines, game) for game in range(offers_to_loop)], 
    axis=0
)

# If team odds today is > 0
if len(nba_team_odds_df) > 0:

    # join nba_game_df
    nba_team_odds_df = nba_team_odds_df.merge(
        nba_game_df[['eventId', 'awayTeamName', 'homeTeamName']],
        on='eventId', how='inner'
    )

    # Create teamType column
    nba_team_odds_df['teamType'] = np.where(
        nba_team_odds_df['label'] == nba_team_odds_df['homeTeamName'],
        'Home',
        'Away'
    )

    # filter for over/under labels (which will always be the same so can just take Over)
    over_under_lines = nba_team_odds_df[nba_team_odds_df['label'].isin(['Over'])][[
        'eventId', 'line']]
    over_under_lines.rename(columns={'line': 'totalPointsLine'}, inplace=True)

    # Get spread lines
    spread_lines = nba_team_odds_df[nba_team_odds_df['oddType'] == 'Spread'][[
        'eventId', 'line', 'teamType']]
    spread_lines.rename(columns={'line': 'spreadLine'}, inplace=True)

    # Filter out Over/Under labels
    nba_team_odds_df = nba_team_odds_df[~nba_team_odds_df['label'].isin(['Over', 'Under'])]

    # Pivot wider on team type and eventId
    nba_team_odds_df = nba_team_odds_df.pivot_table(
        index=['eventId', 'teamType'],
        columns=['oddType'],
        values=['oddsAmerican'],
        aggfunc='first'
    )

    # Fix colummn names then reset index
    nba_team_odds_df.columns = [''.join(col) for col in nba_team_odds_df.columns]
    nba_team_odds_df = nba_team_odds_df.reset_index()

    # Join spread_lines and over_under_lines
    nba_team_odds_df = nba_team_odds_df.merge(spread_lines, on=['eventId', 'teamType'], how='left')
    nba_team_odds_df = nba_team_odds_df.merge(over_under_lines, on='eventId', how='left')

    # Remove American from columns names
    nba_team_odds_df.columns = nba_team_odds_df.columns.str.replace('American', '')

    # Replace plus sign in moneyline with nothing, convert to int
    nba_team_odds_df['oddsMoneyline'] = nba_team_odds_df['oddsMoneyline'].str.replace(
        '+', '').astype(int)

    # Convert spreadLine, oddsSpread, and totalPoints to float
    nba_team_odds_df[['spreadLine', 'oddsSpread', 'totalPointsLine']] = nba_team_odds_df[[
        'spreadLine', 'oddsSpread', 'totalPointsLine']].astype(float)
    
    ## SQL Stored Procedure Scratch for nba_team_odds_df insert/update
    # Params of all columns
    # For each odd column, set value to odds where eventId = eventId and teamType = teamType and odds column IS NOT NULL
    # Break into 4 statements for each odd column, combine into one stored procedure
    ##